In [7]:
import os
import sys

script_path = os.path.dirname(os.path.realpath('__file__'))

sys.path.insert(0, script_path + "/../")

from scrape_articles import _topics_query_,discover_articles
from pprint import pprint
import pandas as pd
import operator

In [2]:
title = _topics_query_(["AAPL", "Apple Inc."])
json = discover_articles(title)

Search Title Query: AAPL%20OR%20Apple%20Inc.


In [3]:
pprint(json)

{'articles': [{'addDate': '2023-06-02T11:11:18.161120+00:00',
               'articleId': 'd6205ae96d28435da708ffecc146e989',
               'authorsByline': '',
               'categories': [{'name': 'Finance'}],
               'claim': '',
               'clusterId': '4687f85c224c4013a95fb5d18bf9ff8c',
               'companies': [{'domains': ['abc.xyz'],
                              'id': '806457af1fd9418db78388760f52c06f',
                              'name': 'Alphabet Inc.',
                              'symbols': ['ABEA.DE',
                                          'ABEC.DE',
                                          'GOOG',
                                          'GOOGL',
                                          'GOOGL.SW']},
                             {'domains': ['apple.com'],
                              'id': '5985872d662c4a3f941a9c581a9fce71',
                              'name': 'Apple Inc.',
                              'symbols': ['AAPL', 'APC.DE', 'APC.F']}]

In [4]:
print(f"Number of articles: {len(json['articles'])}")

Number of articles: 10


In [5]:
json['articles'][0]

{'url': 'https://stocksregister.com/2023/06/02/apple-inc-aapl-worth-considering-for-the-next-few-weeks/',
 'authorsByline': '',
 'articleId': 'd6205ae96d28435da708ffecc146e989',
 'clusterId': '4687f85c224c4013a95fb5d18bf9ff8c',
 'source': {'domain': 'stocksregister.com'},
 'imageUrl': 'https://stocksregister.com/wp-content/uploads/2020/04/31.jpg',
 'country': 'us',
 'language': 'en',
 'pubDate': '2023-06-02T11:00:08+00:00',
 'addDate': '2023-06-02T11:11:18.161120+00:00',
 'refreshDate': '2023-06-02T11:11:18.161122+00:00',
 'score': 110.4862,
 'title': 'Apple Inc. (AAPL) Worth Considering For The Next Few Weeks',
 'description': 'Apple Inc. (NASDAQ:AAPL) concluded the trading at $180.09 on Thursday, June 01 with a rise of 1.60% from its closing price on previous day. Taking a look at stock we notice that its last check on previous day was $177.25 and 5Y monthly beta was reading 1.30 with its price kept floating in the range',
 'content': 'Apple Inc. (NASDAQ:AAPL) concluded the trading a

In [8]:
import operator

def sentiment_bias(sentiment_dict):
    return max(sentiment_dict.items(), key=operator.itemgetter(1))[0]


In [9]:
sentiment_bias(json["articles"][0]["sentiment"])

'negative'

In [10]:
print(f'Title: {json["articles"][0]["title"]}')
print(f"Publish Date: {json['articles'][0]['pubDate'][:10]}")
print(f'Url: {json["articles"][0]["url"]}')
print(f'Sentiment: {json["articles"][0]["sentiment"]}')
# print(f"Content: {json['articles'][0]['content']}")



Title: Apple Inc. (AAPL) Worth Considering For The Next Few Weeks
Publish Date: 2023-06-02
Url: https://stocksregister.com/2023/06/02/apple-inc-aapl-worth-considering-for-the-next-few-weeks/
Sentiment: {'positive': 0.12197371, 'negative': 0.8481135, 'neutral': 0.029912842}


In [21]:
# json interator
def json_article_iterator(json):
    df = pd.DataFrame(columns=['title', 'pubDate', 'url','perigon_sentiment', 'Article'])
    for i, meta in enumerate(json['articles']):
        
        article_df = {}
        article_df['title'] = meta["title"]
        article_df['pubDate'] = meta['pubDate'][:10]
        article_df['url'] = meta['url']
        article_df['perigon_sentiment'] = sentiment_bias(meta["sentiment"])
        article_df['Article'] = meta['content']
        # sentiment_dict = meta["sentiment"]
        new_row = pd.DataFrame(article_df, index=[i])
        
        df = pd.concat([df, new_row], ignore_index=True)
    return df

In [22]:
json_article_iterator(json)

,title,pubDate,url,perigon_sentiment,Article
0,Apple Inc. (AAPL) Worth Considering For The Ne...,2023-06-02,https://stocksregister.com/2023/06/02/apple-in...,negative,Apple Inc. (NASDAQ:AAPL) concluded the trading...
1,Jefferies Maintains Apple (AAPL) Buy Recommend...,2023-06-02,https://www.nasdaq.com/articles/jefferies-main...,positive,"Fintel reports that on June 2, 2023, Jefferies..."
2,Apple Inc. (AAPL) is Attracting Investor Atten...,2023-06-02,https://finance.yahoo.com/news/apple-inc-aapl-...,neutral,Apple (AAPL) has been one of the most searched...
3,Morgan Stanley Maintains Apple (AAPL) Overweig...,2023-06-02,https://www.nasdaq.com/articles/morgan-stanley...,positive,"Fintel reports that on June 2, 2023, Morgan St..."
4,Apple Inc. (AAPL) Plans Major Retail Expansion...,2023-06-02,https://abbonews.com/news/apple-inc-aapl-plans...,neutral,Apple Inc. (NASDAQ: AAPL) has announced plans ...
5,Apple (AAPL) Gains But Lags Market: What You S...,2023-06-02,https://finance.yahoo.com/news/apple-aapl-gain...,negative,Apple (AAPL) closed the most recent trading da...
6,Goldman's new investment list includes Apple (...,2023-06-01,https://www.streetinsider.com/Analyst+Comments...,neutral,Goldman Sachs analysts led by Steven Kron publ...
7,Apple (AAPL) PT Raised to $190 at Morgan Stanl...,2023-06-02,https://web3byts.com/apple-aapl-pt-raised-to-1...,neutral,"Save my name, email, and website in this brows..."
8,Brokers raise Apple (AAPL) stock targets; Jeff...,2023-06-02,https://www.streetinsider.com/Analyst+Comments...,positive,Wall Street analysts are lifting price targets...
9,Insight 2811 Inc. Raises Position in Apple Inc...,2023-06-01,https://www.defenseworld.net/2023/06/01/insigh...,neutral,Insight 2811 Inc. lifted its stake in shares o...


In [25]:
from sentiment_analysis import *

In [26]:
df = json_article_iterator(json)
focus = 'AAPL'

In [27]:
result = generate_sentiment_summary_matrix(df, focus)

Processing article sentiment:   0%|                                                                                                                                                                                                                                                                                                                  | 0/10 [00:00<?, ?it/s]

NEUTRAL


Processing article sentiment:  10%|█████████████████████████████▊                                                                                                                                                                                                                                                                            | 1/10 [00:04<00:42,  4.76s/it]

POSITIVE


Processing article sentiment:  20%|███████████████████████████████████████████████████████████▌                                                                                                                                                                                                                                              | 2/10 [00:09<00:38,  4.80s/it]

...

Processing article summary:  80%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                            | 8/10 [01:01<00:15,  7.56s/it]

The article reports that Wall Street analysts are raising their price targets on Apple (AAPL) shares due to the ongoing AI-led tech frenzy. The article notes that Apple is well-positioned to benefit from the trend towards AI and machine learning, and the


Processing article summary:  90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                              | 9/10 [01:09<00:07,  7.62s/it]

Insight 2811 increased its stake in Apple by 2.6% in Q4 2016, taking its total holdings in the iPhone maker to 10,978 shares. Apple accounts for 0.9% of Insight 281


Processing article summary: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:17<00:00,  7.73s/it]


In [29]:
display(result)

,title,pubDate,url,perigon_sentiment,Article,sentiment,summary
0,Apple Inc. (AAPL) Worth Considering For The Ne...,2023-06-02,https://stocksregister.com/2023/06/02/apple-in...,negative,Apple Inc. (NASDAQ:AAPL) concluded the trading...,NEUTRAL,Apple Inc. (AAPL) saw a 1.60% increase in its ...
1,Jefferies Maintains Apple (AAPL) Buy Recommend...,2023-06-02,https://www.nasdaq.com/articles/jefferies-main...,positive,"Fintel reports that on June 2, 2023, Jefferies...",POSITIVE,Jefferies maintained their Buy recommendation ...
2,Apple Inc. (AAPL) is Attracting Investor Atten...,2023-06-02,https://finance.yahoo.com/news/apple-inc-aapl-...,neutral,Apple (AAPL) has been one of the most searched...,NEUTRAL,Apple's stock has returned 8.6% over the past ...
3,Morgan Stanley Maintains Apple (AAPL) Overweig...,2023-06-02,https://www.nasdaq.com/articles/morgan-stanley...,positive,"Fintel reports that on June 2, 2023, Morgan St...",POSITIVE,"On June 2, 2023, Morgan Stanley maintained its..."
4,Apple Inc. (AAPL) Plans Major Retail Expansion...,2023-06-02,https://abbonews.com/news/apple-inc-aapl-plans...,neutral,Apple Inc. (NASDAQ: AAPL) has announced plans ...,NEUTRAL,Apple has announced plans to expand and revamp...
5,Apple (AAPL) Gains But Lags Market: What You S...,2023-06-02,https://finance.yahoo.com/news/apple-aapl-gain...,negative,Apple (AAPL) closed the most recent trading da...,NEUTRAL,Apple's shares lagged behind the S&P 500's gai...
6,Goldman's new investment list includes Apple (...,2023-06-01,https://www.streetinsider.com/Analyst+Comments...,neutral,Goldman Sachs analysts led by Steven Kron publ...,POSITIVE,"Goldman Sachs has released a new ""Conviction L..."
7,Apple (AAPL) PT Raised to $190 at Morgan Stanl...,2023-06-02,https://web3byts.com/apple-aapl-pt-raised-to-1...,neutral,"Save my name, email, and website in this brows...",IRRELEVANT,"I'm sorry, but it seems that the article you p..."
8,Brokers raise Apple (AAPL) stock targets; Jeff...,2023-06-02,https://www.streetinsider.com/Analyst+Comments...,positive,Wall Street analysts are lifting price targets...,POSITIVE,The article reports that Wall Street analysts ...
9,Insight 2811 Inc. Raises Position in Apple Inc...,2023-06-01,https://www.defenseworld.net/2023/06/01/insigh...,neutral,Insight 2811 Inc. lifted its stake in shares o...,NEUTRAL,Insight 2811 increased its stake in Apple by 2...
